# Analyze the output of Bayesian Fitting
To run this notebook on your MCMC: <br>
1. Update the idbeg and idend values to match the Run folders containing your simulation <br>
2. Update common.jl's datamat to match your mcmc's chain's time span if it doesn't match already
## Load libraries and data

In [ ]:
# Activate the environment
#using Pkg
#Pkg.activate("");
using CSV, DataFrames, Plots, FileIO
pyplot();

# Load Bubar function library
include("common.jl"); include("odesolver.jl"); include("gibbs.jl");

# Load data
idbeg = 1; idend = 2;
M_mcmc,M_err = mergeoutputs(idbeg,idend);
nsmp = length(M_err); @assert nsmp == size(M_mcmc)[2] "Number of samples not consistent";
println("$nsmp samples taken ...");

## Analyze best fit

### Prepare to run simulation

In [ ]:
# Find the best fit
idbest = findfirst(M_err .== minimum(M_err));
sheet = data(); mydat,myaux = csvdat(M_mcmc[:,idbest],sheet.csv_vac,sheet.csv_odh);
sheet = data(mydat); mydep = depmat(mydat,myaux); frc_M = vaxld(); # data used in sim

# Load ODH data
fname = sheet.csv_odh; dfodh = CSV.read(fname,DataFrame);
cols = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80+"];

# Locate the initial and final day in spreadsheet
dates = dfodh[!,:time]; # Vector of dates
day0 = Date("2020-01-01");
dayi = day0 + Day(sheet.tspan[1]);
daym = day0 + Day(sheet.tspan[2]);
dayf = dates[end];
posi = 0; flagfd = false;
while !flagfd
    posi += 1;
    if dates[posi] == dayi
        break
    end
end

if dayf > dates[end]
    posf = length(dates);
else
    posf = 0; flagfd = false;
    while !flagfd
        posf += 1;
        if dates[posf] == dayf
            break
        end
    end
end
ti = getfield(dayi-day0,:value); tf = getfield(dayf-day0,:value); tm = getfield(daym-day0,:value);
plot(dfodh[!,:time],dfodh[!,:daily_confirm],labels="",title="ODH Reported Daily Infections")
vline!([dayi,daym,dayf],labels="")
savefig("odh.pdf")

#### Used to simulate the best fit to the last ODH day, so can compare after the fit to data

In [ ]:
# Comment out if don't want to simulate to last ODH day
mydat[:tspan] = [Float64(ti),Float64(tf)];
sheet = data(mydat); mydep = depmat(mydat,myaux); 
frc_M = (isempty(sheet.csv_vac)) ? [0. 0.;1. 0.] : vaxld(); # data used in sim

### Run the simulation

In [ ]:
# Run the simulation
tpts,ypts = odesolver(sheet,mydep,frc_M);

# Aggregate ypts across unvax, vax, and unwilling for comparing to ODH
Ypts = reshape(ypts,(27,Int64(length(ypts)/(27*length(tpts))),length(tpts)));
U = Ypts[1:9,:,:]; V = Ypts[10:18,:,:]; X = Ypts[19:27,:,:];
Ypts = U + V + X; # Ages x SEIR x Time

# Find daily new infections
#  Use ∫fdx ≈ ∑ᵢfᵢΔxᵢ = L*∑ᵢfᵢ/n
Etot = Ypts[:,2,:];
Zpts = Matrix{Float64}(undef,9,tf-ti);
for i=ti:(tf-1)
    for j=1:9
        val1 = myinterp(tpts,Etot[j,:],Float64(i));
        val2 = myinterp(tpts,Etot[j,:],Float64(i+1));
        
        Zpts[j,i-(ti-1)] = (1/sheet.d_E)*(val1+val2)/2;
    end
end

#  Scale by reporting factor
Zpts *= 1/myaux[:rptλ]*1/myaux[:rptλE];

### Plot a Markov error distribution
#### No moving average

In [ ]:
# Noise = ODH-Model
Noise = hcat(dfodh[!,"0-9"][ti:tf-1] - Zpts[1,:],
         dfodh[!,"10-19"][ti:tf-1] - Zpts[2,:],
         dfodh[!,"20-29"][ti:tf-1] - Zpts[3,:],
         dfodh[!,"30-39"][ti:tf-1] - Zpts[4,:],
         dfodh[!,"40-49"][ti:tf-1] - Zpts[5,:],
         dfodh[!,"50-59"][ti:tf-1] - Zpts[6,:],
         dfodh[!,"60-69"][ti:tf-1] - Zpts[7,:],
         dfodh[!,"70-79"][ti:tf-1] - Zpts[8,:],
         dfodh[!,"80+"][ti:tf-1] - Zpts[9,:]);
Noiseᵗʳ = copy(transpose(Noise));

# Array of Present and Future errors
Mkverr = [Noiseᵗʳ[:,1:end-1][:] Noiseᵗʳ[:,2:end][:]]

# Plot Markov error distribution
histogram2d(Mkverr[:,1],Mkverr[:,2],xlabel="Present Noise",ylabel="Future Noise",title="Noise Distribution about Best Fit")
savefig("mkverr.pdf")

#### Moving average

In [ ]:
# Noise = ODH-Model
Noise = hcat(mymvavg(dfodh[!,"0-9"][ti:tf-1]) - Zpts[1,:],
         mymvavg(dfodh[!,"10-19"][ti:tf-1]) - Zpts[2,:],
         mymvavg(dfodh[!,"20-29"][ti:tf-1]) - Zpts[3,:],
         mymvavg(dfodh[!,"30-39"][ti:tf-1]) - Zpts[4,:],
         mymvavg(dfodh[!,"40-49"][ti:tf-1]) - Zpts[5,:],
         mymvavg(dfodh[!,"50-59"][ti:tf-1]) - Zpts[6,:],
         mymvavg(dfodh[!,"60-69"][ti:tf-1]) - Zpts[7,:],
         mymvavg(dfodh[!,"70-79"][ti:tf-1]) - Zpts[8,:],
         mymvavg(dfodh[!,"80+"][ti:tf-1]) - Zpts[9,:]);
Noiseᵗʳ = copy(transpose(Noise));

# Array of Present and Future errors
Mkverr = [Noiseᵗʳ[:,1:end-1][:] Noiseᵗʳ[:,2:end][:]]

# Plot Markov error distribution
histogram2d(Mkverr[:,1],Mkverr[:,2],xlabel="Present Noise",ylabel="Future Noise",title="Noise Distribution about Best Fit (mvavg)")
savefig("mkverr_mvavg.pdf")

### Plot the solution with ODH data
Immediate next cell computes quantiles of MCMC trajectories without their standard deviation. Switch to markdown and comment out the plot! lines for quantiles in the 2nd next cell if not desired.

In [ ]:
# Load trajectories on all samples for quantiles
qthresh = .95; # Specify middle quantile want
using Statistics
Mtrajs = 0;
qttaxis=0;
for i=idbeg:idend
    df = CSV.read("Run"*string(i)*"/GibbsTrajs.csv",DataFrame,header=false);
    Mtrajs = (i == idbeg) ? convert(Matrix,df[:,2:end]) : [Mtrajs convert(Matrix,df[:,2:end])];
    qttaxis = (i == idend) ? df[!,1] : 0;
end
Mᵗʳ = copy(transpose(Mtrajs)); # copy makes it genuine matrix
dwnsmp = Int64(length(qttaxis)/9);
# Compute the quantiles
Mquants = Matrix{Float64}(undef,9*dwnsmp,2);
for i=1:9*dwnsmp
    Mquants[i,:] = [quantile!(Mᵗʳ[:,i],(1-qthresh)/2) quantile!(Mᵗʳ[:,i],1-(1-qthresh)/2)];
end
quantlow = reshape(Mquants[:,1],dwnsmp,9); quanthigh = reshape(Mquants[:,2],dwnsmp,9);
mycolor = :red; # Color for MCMC bands

In [ ]:
# Plot the data
ymax = maximum(convert(Matrix,dfodh[ti:tf,2:end-2])) + 10;
p1 = plot(ti:tf,dfodh[!,"0-9"][ti:tf],labels="ODH",title="0-9"); plot!(ti:tf,mymvavg(dfodh[!,"0-9"][ti:tf]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[1,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); 
quantmean = .5*quantlow[:,1] + .5*quanthigh[:,1]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,1],quanthigh[:,1]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tm],labels="");

p2 = plot(ti:tf,dfodh[!,"10-19"][ti:tf],labels="ODH",title="10-19"); plot!(ti:tf,mymvavg(dfodh[!,"10-19"][ti:tf]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[2,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,2] + .5*quanthigh[:,2]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,2],quanthigh[:,2]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tm],labels="");

p3 = plot(ti:tf,dfodh[!,"20-29"][ti:tf],labels="ODH",title="20-29"); plot!(ti:tf,mymvavg(dfodh[!,"20-29"][ti:tf]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[3,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,3] + .5*quanthigh[:,3]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,3],quanthigh[:,3]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tm],labels="");

p4 = plot(ti:tf,dfodh[!,"30-39"][ti:tf],labels="ODH",title="30-39"); plot!(ti:tf,mymvavg(dfodh[!,"30-39"][ti:tf]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[4,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,4] + .5*quanthigh[:,4]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,4],quanthigh[:,4]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tm],labels="");

p5 = plot(ti:tf,dfodh[!,"40-49"][ti:tf],labels="ODH",title="40-49"); plot!(ti:tf,mymvavg(dfodh[!,"40-49"][ti:tf]),labels="ODH Smooth");# plot!(ti:(tf-1),Zpts[5,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,5] + .5*quanthigh[:,5]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,5],quanthigh[:,5]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tm],labels="");

p6 = plot(ti:tf,dfodh[!,"50-59"][ti:tf],labels="ODH",title="50-59"); plot!(ti:tf,mymvavg(dfodh[!,"50-59"][ti:tf]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[6,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,6] + .5*quanthigh[:,6]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,6],quanthigh[:,6]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tm],labels="");

p7 = plot(ti:tf,dfodh[!,"60-69"][ti:tf],labels="ODH",title="60-69"); plot!(ti:tf,mymvavg(dfodh[!,"60-69"][ti:tf]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[7,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,7] + .5*quanthigh[:,7]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,7],quanthigh[:,7]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tm],labels="");

p8 = plot(ti:tf,dfodh[!,"70-79"][ti:tf],labels="ODH",title="70-79"); plot!(ti:tf,mymvavg(dfodh[!,"70-79"][ti:tf]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[8,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,8] + .5*quanthigh[:,8]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,8],quanthigh[:,8]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tm],labels="");

p9 = plot(ti:tf,dfodh[!,"80+"][ti:tf],labels="ODH",title="80+"); plot!(ti:tf,mymvavg(dfodh[!,"80+"][ti:tf]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[9,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,9] + .5*quanthigh[:,9]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,9],quanthigh[:,9]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tm],labels="");


println("Dates run from $dayi to $dayf while simulated up to $daym")
lay = @layout [a b c; d e f; g h i];
plot(p1,p2,p3,p4,p5,p6,p7,p8,p9, layout=lay, size = (850,425))
savefig("prediction.pdf")

In [ ]:
ENV["COLUMNS"] = 1000; ENV["ROWS"] = 1000;
println("Best parameters:")
df = DataFrame(Dict{String,Float64}("L1 Err"=>M_err[idbest],"r0"=>M_mcmc[12,idbest],
                                    "α"=>M_mcmc[31,idbest],"ω"=>M_mcmc[32,idbest],
                                    "rptλ"=>M_mcmc[176,idbest],"bayσ"=>M_mcmc[177,idbest],
                                    "vι0"=>M_mcmc[179,idbest],
                                    "rptλE"=>M_mcmc[180,idbest],"rptλI"=>M_mcmc[181,idbest],
                                    "Δt"=>M_mcmc[182,idbest],"r0λ"=>M_mcmc[183,idbest]))

## Collect marginal distributions

In [ ]:
p1 = histogram(M_mcmc[12,:],normalize=:probability, labels="", title="r0"); #r0
p2 = histogram(M_mcmc[31,:],normalize=:probability, labels="", title="α"); #α
p3 = histogram(M_mcmc[32,:],normalize=:probability, labels="", title = "ω"); #ω
p4 = histogram(M_mcmc[176,:],normalize=:probability, labels="", title="rptλ"); #rptλ
p5 = histogram(M_mcmc[177,:],normalize=:probability, labels="", title="bayσ"); #bayσ
#p6 = histogram(M_err,normalize=:probability, labels="", title="MSE"); #L1 Err
p7 = histogram(M_mcmc[179,:],normalize=:probability,labels="",title="vι0"); #vι0
#p8 = histogram(M_mcmc[180,:],normalize=:probability,labels="",title="rptλE"); #rptλE
#p9 = histogram(M_mcmc[181,:],normalize=:probability,labels="",title="rptλI"); #rptλI
#p10 = histogram(M_mcmc[182,:],normalize=:probability,labels="",title="Δt"); #Δt
#p11 = histogram(M_mcmc[183,:],normalize=:probability,labels="",title="r0λ"); #r0λ

lay = @layout [a b;c d; e f];
plot(p2,p3,p1,p4,p5,p7, layout=lay, size=(850,425))
savefig("marginals.pdf")

## Trace Plots

In [ ]:
nstep = 50;

q1 = plot(M_mcmc[12,1:nstep:end],labels="", title="r0"); #r0
q2 = plot(M_mcmc[31,1:nstep:end],labels="", title="α"); #α
q3 = plot(M_mcmc[32,1:nstep:end],labels="", title = "ω"); #ω
q4 = plot(M_mcmc[176,1:nstep:end],labels="", title="rptλ"); #rptλ
q5 = plot(M_mcmc[177,1:nstep:end],labels="", title="bayσ"); #bayσ
#q6 = plot(M_err,labels="",title="MSE")
q7 = plot(M_mcmc[179,1:nstep:end],labels="",title="vι0"); #vι0
#q8 = plot(M_mcmc[180,:],labels="",title="rptλE"); #rptλE
#q9 = plot(M_mcmc[181,:],labels="",title="rptλI"); #rptλI
#q10 = plot(M_mcmc[182,:],labels="",title="Δt"); #Δt
#q11 = plot(M_mcmc[183,:],labels="",title="r0λ"); #r0λ

lay = @layout [a b;c d; e f];
plot(q2,q3,q1,q4,q5,q7, layout=lay, size=(850,425))
savefig("trace.pdf")